This notebook will use the robin_stocks.robinhood.export functions which write all orders to a csv file.  We will do this for stock, options and crypto orders

Install the required libraries

In [5]:
%pip install robin_stocks pandas numpy ipykernel


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Log on to the robinhood API endpoint, and create the output directory if it doesn't exist

In [7]:
import robin_stocks.robinhood as r
import pandas as pd
import os  
# Prompt for email address
email = input("Please enter your Robinhood email address: ")

login = r.login(email)

# Ensure the output directory exists
output_dir = '../output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir) 


Now that we are logged in lets get all the stock orders

In [9]:
# # Get all stock orders

r.export_completed_stock_orders('/', file_name='../output/stock_output.csv')

Found Additional pages.
Loading page 2 ...
Loading page 3 ...


Now that we have the stock orders lets parse them, caclute cost of each order, and sum up the total for cumulative stock profit and loss.  Stock buys will be updated with a negative cost and sells will be a positive cost.   For example I buy 10 shares of AMD for $10 a share, this will cost me $100 and will reduce my balance by that much.  Then if I decide to sell my 10 AMD shares and get filled at $11 share for a total of $110, which will be added to my balance.  The total P&L for that trade would be +$10 since I sold the position for $10 more than I paid for it.  Conversly if I sold the 10 AMD and got filled at $9 a share for a total of $90, which will be added back to my cash balance.  The total P&L for this trade would be -$10 since I sold the position for $10 less than I paid for it.

This may have option exercise events in it, which we will need to account for in the total P&L. We will need to account for these events in the total P&L calculation.  THis may require pulling historical option pricing data and calculating the P&L based on the bid/ask at the time of the event.  This is not completed yet.

In [16]:
import pandas as pd

# Load your spreadsheet
df = pd.read_csv('../output/stock_output.csv')

# Convert the 'date' column to datetime
df['date'] = pd.to_datetime(df['date'],  errors='coerce', utc=True)

#df = df[df['date'] >= '2019-02-12']

# Sort the DataFrame based on the date column
df = df.sort_values(by=['symbol','date'])

# Assuming 'quantity', 'average_price', and 'type' columns exist
df['cost'] = df['quantity'] * df['average_price']

# Adjust the cost based on the type of order
df.loc[df['side'] == 'buy', 'cost'] *= -1

# print(df)
total_stock_cost = df['cost'].sum()
print(total_stock_cost)

df.to_csv('../output/processed_stock_orders.csv', index=False)


3296.7660999999985


Now lets get all crypto orders

In [8]:
# # Get all crypto orders
r.export_completed_crypto_orders('/', file_name='../output/crypto_output.csv')

Now lets do the same thing we did with the stock orders to the crypto orders

In [17]:
import pandas as pd

# Load your spreadsheet
df = pd.read_csv('../output/crypto_output.csv')

# Convert the 'date' column to datetime
df['date'] = pd.to_datetime(df['date'],  errors='coerce', utc=True)

#df = df[df['date'] >= '2019-02-12']

# Sort the DataFrame based on the date column
df = df.sort_values(by=['symbol','date'])

# Assuming 'quantity', 'average_price', and 'type' columns exist
df['cost'] = df['quantity'] * df['average_price']

# Adjust the cost based on the type of order
df.loc[df['side'] == 'buy', 'cost'] *= -1

print(df)
total_crypto_cost = df['cost'].sum()
print(total_crypto_cost)

df.to_csv('../output/processed_crypto_orders.csv', index=False)


     symbol                             date order_type  side  fees  \
22   BTCUSD 2018-08-05 06:49:50.771000+00:00     market   buy   0.0   
21   BTCUSD 2018-08-12 07:49:48.270000+00:00     market  sell   0.0   
9    BTCUSD 2023-12-14 15:06:24.232000+00:00     market  sell   0.0   
8    BTCUSD 2024-06-24 19:04:34.507719+00:00     market   buy   0.0   
7    BTCUSD 2024-07-04 14:30:08.042036+00:00     market   buy   0.0   
6    BTCUSD 2024-07-05 02:53:50.275000+00:00      limit   buy   0.0   
3    BTCUSD 2024-08-12 23:20:08.899000+00:00     market  sell   0.0   
15  DOGEUSD 2018-09-09 02:23:25.038000+00:00     market   buy   0.0   
14  DOGEUSD 2018-09-10 16:45:36.039000+00:00     market  sell   0.0   
11  DOGEUSD 2018-10-09 11:18:51.460000+00:00     market   buy   0.0   
10  DOGEUSD 2018-10-09 11:19:30.680000+00:00     market  sell   0.0   
34   ETHUSD 2018-06-06 20:23:57.487000+00:00      limit   buy   0.0   
33   ETHUSD 2018-06-29 20:23:59.053000+00:00      limit   buy   0.0   
32   E

Now lets get all the options orders.

In [21]:
# # Get all option orders
r.export_completed_option_orders('/', file_name='../output/options_output.csv')

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...


This groups spread order by order_created_at and then calculates total cost based on debit and credit direction.  Spreads orders show up multiple time (number of legs) with the total price and quantity listed on each order, so we group them and only sum them up once per opening and closing spread order

In [22]:
import pandas as pd

# Load your spreadsheet
df = pd.read_csv('../output/options_output.csv')

# Convert 'order_created_at' to datetime
df['order_created_at'] = pd.to_datetime(df['order_created_at'])
df['order_created_at'] = df['order_created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Filter for the year 2024
#df = df[df['order_created_at'].str.startswith('2024')]

# Select only the columns we want
df = df[['order_created_at', 'chain_symbol', 'expiration_date', 
         'strike_price', 'option_type', 'direction', 'order_quantity', 
         'order_type', 'opening_strategy', 'closing_strategy', 'price', 'processed_quantity']]

# Group by 'order_created_at' and aggregate
aggregated_df = df.groupby('order_created_at').agg({
    'chain_symbol': 'first',
    'expiration_date': 'first',
    'strike_price': lambda x: list(sorted(x, key=abs, reverse=True)),  # Create a list of strike price
    'option_type': 'first',
    'direction': 'first',
    'opening_strategy': 'first',
    'closing_strategy': 'first',
    'price': 'first',
    'order_quantity': 'first'
}).reset_index()

# Sort the DataFrame
aggregated_df = aggregated_df.sort_values(by=['chain_symbol', 'expiration_date'])

# Calculate cost
aggregated_df['cost'] = aggregated_df['price'] * aggregated_df['order_quantity'] * 100

# Adjust the cost based on the direction
aggregated_df['cost'] = aggregated_df.apply(
    lambda x: -x['cost'] if x['direction'] == 'debit' else x['cost'],
    axis=1
)      

total_option_cost = aggregated_df['cost'].sum()
print(f"Total option cost: ${total_option_cost:.2f}")
# print(aggregated_df['order_created_at'].count())
# print(aggregated_df['order_quantity'].sum())

# Save the results
aggregated_df.to_csv('../output/options_orders_parsed.csv', index=False)

# Count rows where 'opening_strategy' is not None
total_opening_strategy_count = aggregated_df['opening_strategy'].notna().sum()
total_closing_strategy_count = aggregated_df['closing_strategy'].notna().sum()


# # Count rows where 'closing_strategy' contains 'spread' or 'iron'
# spread_closing_count = df['closing_strategy'].str.contains('spread|iron', case=False, na=False).sum()

# # Count rows where 'opening_strategy' contains 'spread' or 'iron'
# spread_opening_count = df['opening_strategy'].str.contains('spread|iron', case=False, na=False).sum()

# # Count rows where 'closing_strategy' contains 'spread' or 'iron'
# non_spread_closing_count = total_closing_strategy_count - spread_closing_count

# # Count rows where 'opening_strategy' contains 'spread' or 'iron'
# non_spread_opening_count = total_opening_strategy_count - spread_opening_count
# Calculate total order quantity for opening_strategy
total_opening_quantity = aggregated_df['order_quantity'][aggregated_df['opening_strategy'].notna()].sum()
print(f"Total opening strategy quantity: {total_opening_quantity}")

# Calculate total order quantity for closing_strategy
total_closing_quantity = aggregated_df['order_quantity'][aggregated_df['closing_strategy'].notna()].sum()
print(f"Total closing strategy quantity: {total_closing_quantity}")


#Print the counts
print(f"Opening strategy rows: {total_opening_strategy_count}")
print(f"Closing strategy rows: {total_closing_strategy_count}")
# print(f"Opening strategy rows with 'spread' or 'iron': {spread_opening_count}")
# print(f"Closing strategy rows with 'spread' or 'iron': {spread_closing_count}")
# print(f"Opening strategy rows not with 'spread' or 'iron': {non_spread_opening_count}")
# print(f"Closing strategy rows not with 'spread' or 'iron': {non_spread_closing_count}")

Total option cost: $-3247.00
Total opening strategy quantity: 1008.0
Total closing strategy quantity: 618.0
Opening strategy rows: 544
Closing strategy rows: 335


This gets all the options events for exercise and assignment calculation

In [59]:
# ... existing code ...
import pandas as pd
import json  # Use json to parse the string representation of lists/dictionaries

unique_symbols = aggregated_df['chain_symbol'].unique()
total_option_event_cost = 0  # Initialize total cash amount

# Create a list to store event data
event_data = []

for symbol in unique_symbols:
    events = r.get_events(symbol)  # Fetch events for the symbol
    
    for event in events:
        # Ignore rows with total_cash_amount = 0
        if float(event['total_cash_amount']) == 0 :
            continue

        # print(event)
        # Append event details to the list
        event_data.append({
            'created_at': event['created_at'],
            'chain_symbol': symbol,
            'direction': event['direction'],
            'type': event['type'],
            'quantity': event['quantity'],
            'total_cash_amount': event['total_cash_amount'],
            'state': event['state'],
            'underlying_price': event['underlying_price']
        })
        if event['type'] == 'exercise':
            total_option_event_cost += float(event['total_cash_amount'])  # Add for exercise
        elif event['type'] == 'credit':
            total_option_event_cost -= float(event['total_cash_amount']) 
    #     elif event['direction'] == 'credit' and event['type'] == 'assignment':
    #         total_option_event_cost -= float(event['total_cash_amount'])  
    #     elif event['direction'] == 'debit' and event['type'] == 'exercise':
    #         total_option_event_cost += float(event['total_cash_amount'])  # Subtract for assignment

    
# Create a DataFrame from the event data
events_df = pd.DataFrame(event_data)

# Save the events DataFrame to a new CSV file
events_df.to_csv('../output/option_events.csv', index=True)


# Print the total cash amount
print(f"Total cash amount: {total_option_event_cost}")


Total cash amount: 7540137.360000001


In [60]:
# ... existing code ...
import pandas as pd
import json  # Use json to parse the string representation of lists/dictionaries

unique_symbols = aggregated_df['chain_symbol'].unique()
total_option_credits = 0
total_option_debits = 0
total_option_exercise = 0 
total_option_assignment = 0  

# Read the raw event data correctly
raw_event_data = pd.read_csv('../output/option_events_raw.csv')


# Create a list to store event data
event_data = []

# Iterate over the DataFrame rows
for index, event in raw_event_data.iterrows():  
    event_data.append({
        # 'symbol': event['equity_components']['symbol'],
        # 'price': event['equity_components']['price'],
        # 'quantity': event['equity_components']['quantity'],
        # 'side': event['equity_components']['side'],
        'created_at': event['created_at'],
        'direction': event['direction'],
        'quantity': event['quantity'],
        'total_cash_amount': event['total_cash_amount'],
        'state': event['state'],
        'underlying_price': event['underlying_price']
    })
    if event['direction'] == 'credit' and event['source_ref_id'] :
        total_option_credits += float(event['total_cash_amount'])  # Add for exercise
    elif event['direction'] == 'debit' and event['source_ref_id'] != '' :
        total_option_debits -= float(event['total_cash_amount'])  # Subtract for assignment
    
    if event['type'] == 'exercise':
        total_option_exercise += float(event['total_cash_amount'])  # Add for exercise
    elif event['type'] == 'assignment':
        total_option_assignment -= float(event['total_cash_amount']) 
    
# Create a DataFrame from the event data
events_df = pd.DataFrame(event_data)
# Save the events DataFrame to a new CSV file
events_df.to_csv('../output/option_events.csv', index=True)
# Print the total cash amount
print(f"Total option events {len(events_df)}")
print(f"Total option event credits: {total_option_credits}")
print(f"Total option event debits: {total_option_debits}")
print(f"Total options exercised: {total_option_exercise}")
print(f"Total options assigned: {total_option_assignment}")

print(f" exercise + assignment {total_option_exercise + total_option_assignment}")
print(f" credits + debits {total_option_credits + total_option_debits}")

print(f" exercised - credits {total_option_exercise - total_option_credits}")
print(f" assigned - debits { total_option_assignment - total_option_debits}")

total_option_event_cost = total_option_exercise - total_option_credits


Total option events 52
Total option event credits: 7529537.360000001
Total option event debits: -7566000.0
Total options exercised: 7540137.360000001
Total options assigned: -7555400.0
 exercise + assignment -15262.639999998733
 credits + debits -36462.63999999873
 exercised - credits 10600.0
 assigned - debits 10600.0


In [69]:
print(f"Total stock cost: ${total_stock_cost:.2f}")
print(f"Total crypto cost: ${total_crypto_cost:.2f}")
print(f"Total option cost: ${total_option_cost:.2f}")
print(f"Total option event cost: ${total_option_event_cost:.2f}")


Total stock cost: $3296.77
Total crypto cost: $-153.82
Total option cost: $-3247.00
Total option event cost: $10600.00
